# Maximum Likelihood Estimation with Bernoulli
### 1.standardized logrithmic liklihood
This code estimates parameters ($p$) by maximizing the standardized (by T) log likelihood of an IID normal sample:

\begin{equation*}
Q_T(\theta) = \frac{1}{T}\log(L(\{x\},p))
\end{equation*}
\begin{equation*}
=\frac{1}{T}\sum_{t=1}^T(x_tlog(1-p)+(1-x_t)log(1-p))
\end{equation*}
\begin{equation*}
= log(1-p)+\log(\frac{p}{1-p})*\frac{1}{T}\sum_{t=1}^T x_t.
\end{equation*}

In [2]:
import os
import pandas as pd
import numpy as np
import scipy.optimize
from platform import python_version

In [3]:
# The recommended python version is 3.8 or 3.9
print(python_version())
# Check current directory
os.getcwd()

3.10.12


'/content'

## Bernoulli sample generation

In [4]:
# The number of observations (the sample size)
T = 10000
p=0.3 # we set 0.3 as p
# The true parameters
mu = p
sigma2 = p*(1-p)
sigma = np.sqrt(sigma2)

# An iid sample from the Bernoulli random variable with p
np.random.seed(42)# set seed to keep sample unchanged
x = np.random.choice([0, 1], size=T, p=[1 - p, p])

## 2.(1) $1/T$ log-likelihood function

In [5]:
def loglik(parameter, x):

    lik = np.log(1-parameter) + np.log(parameter/(1-parameter)) * np.mean(x)
    # Below, we take the likelihood with a negative sign since fmin minimizes but we need to maximize

    f = -lik
    return f

## 2.(2) Estimation the Parameter
### We compare `naive estimates` (the sample mean and variance from the data) to the `maximum likelihood` estimates.

\begin{equation*}
Q_T(\theta) = \frac{1}{T}\log(L(\{x\},p))= log(1-p)+\log(\frac{p}{1-p})*\frac{1}{T}\sum_{t=1}^T x_t.
\end{equation*}

In [6]:
########################
# The naive estimates
########################
meanx = np.mean(x)
varx = np.var(x, ddof=1)
px = np.count_nonzero(x)/T

###################
# MLE
###################

# Initial guess
parameter_guess = [0.5]

# Maximizing the likelihood

estimate = scipy.optimize.fmin(func=loglik,
                                x0=parameter_guess,
                                args=(x,),
                                xtol=1e-5,
                                ftol=1e-5,
                                disp=0)
estimate=float(estimate[0])
# Compare results
print('Note: The naive variance estimate is the sample variance divided by T-1 (rather than by T) for unbiasedness.')
table = pd.DataFrame({'True': [mu, sigma2, p], 'Naive': [meanx, varx, px], 'MLE': [estimate, estimate*(1-estimate), estimate]}, index=['Mean','Variance','P'])
table

Note: The naive variance estimate is the sample variance divided by T-1 (rather than by T) for unbiasedness.


,True,Naive,MLE
Mean,0.30,0.288700,0.288702
Variance,0.21,0.205373,0.205353
P,0.30,0.288700,0.288702


## 2.(3) Asymptotic standard error

We know that the ML estimator is asymptotically normal:
\begin{equation*}
\sqrt{T}\left( \widehat{\theta }_{MLE} -\theta _{0}\right) \overset{d}{\rightarrow} N(0,\Omega_{0}^{-1})
\end{equation*}
with $$\Omega_0 = \mathbb{E}\left( \frac{\partial \log p(x_{t}|x_{t-1},...,\theta _{0})}{%
\partial \theta }\frac{\partial \log p(x_{t}|x_{t-1},...,\theta _{0})}{%
\partial \theta ^{^{\top }}}\right).$$

Alternatively, we can also write
\begin{equation*}
\sqrt{T}\left( \widehat{\theta }_{MLE} -\theta _{0}\right) \overset{d}{\rightarrow} N(0,-B_{0}^{-1})
\end{equation*}
with $$B_0 = \mathbb{E}\left(\frac{\partial ^{2}\log p(x_{t}|x_{t-1},...,\theta
_{0})}{\partial \theta \partial \theta ^{^{\top }}}\right),$$

since $$\Omega_0 = - B_0.$$



In essence, we have two ways to compute the asymptotic variance. The `first method` uses the gradient, i.e., the first derivative of the log conditional probability. (In our case the conditional probability is the same as the unconditional probability because the data are IID.)

\begin{equation*}
\widehat{\mathbb{V}}(\widehat{\theta}_{MLE})=\frac{1}{T}\left(\frac{1}{T}\sum_{t=1}^{T}\frac{\partial \log p(x_{t},\widehat{\theta} _{MLE})}{%
\partial \theta }\frac{\partial \log p(x_{t},\widehat{\theta} _{MLE})}{%
\partial \theta ^{^{\top }}}\right)^{-1},
\end{equation*}
where
\begin{eqnarray*}
\frac{\partial \log p(x_{t},\widehat{\theta} _{MLE})}{\partial \theta } = \frac{\partial (\log (1-p)+x_tlog(\frac{p}{1-p}))}{\partial p }
= \frac{x_t-p}{p(1-p)}
\end{eqnarray*}

The `second method` uses the Hessian, i.e., the second derivative of the log conditional probability.

\begin{equation*}
\widehat{\mathbb{V}}(\widehat{\theta}_{MLE})=-\frac{1}{T}\left(\frac{1}{T}\sum_{t=1}^{T}\frac{\partial ^{2}\log p(x_{t},\widehat{\theta}_{MLE})}{\partial \theta \partial \theta ^{^{\top }}}\right)^{-1},
\end{equation*}
where
\begin{eqnarray*}
\frac{\partial^2 \log p(x_{t},\widehat{\theta} _{MLE})}{\partial \theta \partial \theta^{\top}} = \frac{\frac{x_t-p}{p(1-p)}}{\partial p}=
\frac{-p(1-p)-(x_t-p)(1-2p)}{p^2(1-p)^2}
\end{eqnarray*}



#### Method 1: We compute the first derivative (the gradient).

In [10]:
Omega_hat = 0    # Initialize the Omega

for t in range(T):
    # compute the gradient for each time t
    gradient = (x[t] - estimate) / (estimate*(1-estimate))
    # Now we sum up the gradient^2 dividend by T
    Omega_hat = Omega_hat + (gradient**2/T)

VarCov = (1 / T) * (1/Omega_hat)      # Invert Omega_hat and divide by T to compute the asymptotic variance
stderror_mle = np.sqrt(VarCov)                  # std errors are square roots of the variances

#### Method 2: We compute the second derivative (the Hessian)

In [11]:
B_hat = 0       # Initialize the B

for t in range(T):
    # compute the Hessian for each time t
    hessian = (-estimate*(1-estimate)-(x[t] - estimate)*(1-2*estimate)) / ((estimate ** 2)*((1-estimate)**2))
    # Now we sum up the Hessian divided by T
    B_hat = B_hat + (hessian /T)

VarCov2 = - (1 / T) * (1/(B_hat))       # Invert B_hat and divide by T to compute the asymptotic variance
stderror_mle2 = np.sqrt(VarCov2)                # std errors are square roots of the variances

### The results

In [12]:
# compare the results between methods
table = pd.DataFrame({'True': p, 'Naive': px, 'MLE': estimate,
            'Std Errors with Omega': stderror_mle, 'Std Errors with B': stderror_mle2},index=['P'])
table

,True,Naive,MLE,Std Errors with Omega,Std Errors with B
P,0.3,0.2887,0.288702,0.004532,0.004532
